In [1]:
# Connect Driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [28]:
CSV_HEADER=['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label', 'attack_cat'] # bo Src_IP, Dst_IP, Timestamp, {'Flow ID', 'Src Port', 'Dst Port', 'Protocol'}
CSV_HEADER_WITHOUT_LABEL = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

# Bảng mã hóa
encoded_to_original = {
    0: 'Benign',
    1: 'Analysis',
    2: 'Backdoor',
    3: 'DoS',
    4: 'Exploits',
    5: 'Fuzzers',
    6: 'Generic',
    7: 'Reconnaissance',
    8: 'Shellcode',
    9: 'Worms'
}
# decoded_labels = [encoded_to_original[label] for label in y_pred]
# decoded_labels

In [4]:
!pip install flwr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.0
    Uninstalling typer-0.15.0:
      Successfully uninstalled typer-0.15.0


In [29]:
df = pd.read_csv("/content/drive/MyDrive/BT3/CIC_UNSW-LABEL_Attack_cat.csv")
df = df.drop(['attack_cat'], axis=1)

In [30]:
scaler = StandardScaler()
scaler.fit(df[CSV_HEADER_WITHOUT_LABEL])
df[CSV_HEADER_WITHOUT_LABEL] = scaler.transform(df[CSV_HEADER_WITHOUT_LABEL])

In [31]:
# Chia dữ liệu thành train/test
train_data, test_data = train_test_split(df, test_size=0.2)
train_data_1, train_data_2 = train_test_split(train_data, test_size=0.5)

# Lưu dữ liệu vào các tệp
train_data_1.to_csv("train_data_1.csv", index=False, header=True)
train_data_2.to_csv("train_data_2.csv", index=False, header=True)
train_data.to_csv("train_data.csv", index=False, header=True)
test_data.to_csv("test_data.csv", index=False, header=True)


In [33]:
# Xác định đặc trưng (X) và nhãn (y)
X_train_1 = train_data_1[CSV_HEADER_WITHOUT_LABEL]
y_train_1 = train_data_1['Label']

X_train_2 = train_data_2[CSV_HEADER_WITHOUT_LABEL]
y_train_2 = train_data_2['Label']

X_test = test_data[CSV_HEADER_WITHOUT_LABEL]
y_test = test_data['Label']


In [12]:
# Huấn luyện mô hình Logistic Regression cho client 1 và client 2
model_1 = LogisticRegression(max_iter=1000)
model_1.fit(X_train_1, y_train_1)

model_2 = LogisticRegression(max_iter=1000)
model_2.fit(X_train_2, y_train_2)


LogisticRegression(max_iter=1000)

In [15]:
# Tính kích thước của dữ liệu mỗi client
size_train_1 = len(X_train_1)
size_train_2 = len(X_train_2)

# Tính trọng số của mỗi client dựa trên kích thước dữ liệu
total_size = size_train_1 + size_train_2
weight_1 = size_train_1 / total_size
weight_2 = size_train_2 / total_size

# Lấy các trọng số từ mô hình của mỗi client
coef_1 = model_1.coef_
intercept_1 = model_1.intercept_

coef_2 = model_2.coef_
intercept_2 = model_2.intercept_

# Tổng hợp trọng số theo công thức Weighted Federated Averaging
avg_coef = weight_1 * coef_1 + weight_2 * coef_2
avg_intercept = weight_1 * intercept_1 + weight_2 * intercept_2

# Tạo mô hình mới với trọng số tổng hợp
model_avg = LogisticRegression()
model_avg.coef_ = avg_coef
model_avg.intercept_ = avg_intercept
# Added line below to fix the error
model_avg.classes_ = model_1.classes_ # or model_2.classes_, they should be the same


In [17]:
# Dự đoán và đánh giá mô hình trên tập test
y_pred = model_avg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Federated Averaging model: {accuracy}")

Accuracy of Federated Averaging model: 0.8983847381757699


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Dự đoán kết quả từ mô hình tổng hợp trên tập kiểm tra
y_pred = model_avg.predict(X_test)

# In ra một số thông tin cơ bản
print(f"Predictions: {y_pred[:10]}")  # In 10 kết quả đầu tiên


Predictions: [0 7 5 0 0 0 0 0 5 0]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Dự đoán với mô hình tổng hợp
y_pred = model_avg.predict(X_test)

# Đánh giá hiệu suất mô hình
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1 = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

# In ra các chỉ số đánh giá
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

# Tính Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# In báo cáo phân loại (classification report)
report = classification_report(y_test, y_pred, target_names=encoded_to_original.values())
print("\nClassification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 89.84%
Precision: 0.92
Recall: 0.90
F1-score: 0.90

Confusion Matrix:
[[69755     0     0     2   235  1528     7   140     0     0]
 [    0     0     0     0    66     0     1     0     0     0]
 [    2     0     5     0    33    54     1     4     0     0]
 [   11     0     0    20   443   319     4    52     0     0]
 [   53     0     0     9  3906  2112    56   103     0     0]
 [   19     0     0     0   469  4670    14   702     0     0]
 [   14     0     0     2   338   320   224    33     0     0]
 [  106     0     0     0    32  1318     0  1900     0     0]
 [    2     0     0     0    11   327     0   108     0     0]
 [    0     0     0     0    22    29     0     2     0     0]]

Classification Report:
                 precision    recall  f1-score   support

        Benign       1.00      0.97      0.99     71667
      Analysis       0.00      0.00      0.00        67
      Backdoor       1.00      0.05      0.10        99
           DoS       0.61      0.02    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
